In [1]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/home-credit-default-risk'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv(path + '/train.csv').set_index('SK_ID_CURR')
test = pd.read_csv(path + '/test.csv').set_index('SK_ID_CURR')

In [ ]:
train.loc[:,['b_now_overdue_percent_Credit card', 'b_now_overdue_percent_Consumer credit', 'b_now_overdue_percent_Car loan',
             'b_now_overdue_percent_all']] = train[['b_now_overdue_percent_Credit card', 'b_now_overdue_percent_Consumer credit', 'b_now_overdue_percent_Car loan',
             'b_now_overdue_percent_all']].replace([np.inf], 0)
test.loc[:,['b_now_overdue_percent_Credit card', 'b_now_overdue_percent_Consumer credit', 'b_now_overdue_percent_Car loan',
             'b_now_overdue_percent_all']] = test[['b_now_overdue_percent_Credit card', 'b_now_overdue_percent_Consumer credit', 'b_now_overdue_percent_Car loan',
             'b_now_overdue_percent_all']].replace([np.inf], 0)      

In [ ]:
train_cash = train[train['NAME_CONTRACT_TYPE'] == 'Cash loans']
y_train_cash = train_cash['TARGET']
train_cash = train_cash.drop(columns=['TARGET', 'NAME_CONTRACT_TYPE'])
test_cash = test[test['NAME_CONTRACT_TYPE'] == 'Cash loans']
y_test_cash = test_cash['TARGET']
test_cash = test_cash.drop(columns=['TARGET', 'NAME_CONTRACT_TYPE'])
train_rev = train[train['NAME_CONTRACT_TYPE'] == 'Revolving loans']
y_train_rev = train_rev['TARGET']
train_rev = train_rev.drop(columns=['TARGET', 'NAME_CONTRACT_TYPE'])
test_rev = test[test['NAME_CONTRACT_TYPE'] == 'Revolving loans']
y_test_rev = test_rev['TARGET']
test_rev = test_rev.drop(columns=['TARGET', 'NAME_CONTRACT_TYPE'])


## Cash loans

### Предворительная обработка данных (выделение категориальных и интервальных переменных, разбиение интервальных переменных на бины, обработка пропусков)

In [ ]:
categorical = []
for c in train_cash.columns:
  if train_cash[c].dtype == 'object':
    categorical.append(c)
categorical += ['FLAG_MOBIL', 'FLAG_EMP_PHONE',	'FLAG_WORK_PHONE',	'FLAG_CONT_MOBILE',
               'FLAG_PHONE', 'FLAG_EMAIL', 'REG_REGION_NOT_LIVE_REGION',
               'REG_REGION_NOT_WORK_REGION',	'LIVE_REGION_NOT_WORK_REGION',	'REG_CITY_NOT_LIVE_CITY',	'REG_CITY_NOT_WORK_CITY',
               'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'b_Microloan', 'b_overdue_all',
               'b_overdue_Credit card', 'b_overdue_Consumer credit', 'b_overdue_Mortgage', 'b_overdue_Car loan', 'b_overdue_all_1',
               'b_overdue_Credit card_1', 'b_overdue_Consumer credit_1', 'b_overdue_Mortgage_1', 'b_overdue_Car loan_1', 'b_overdue_all_3', 
               'b_overdue_Credit card_3', 'b_overdue_Consumer credit_3', 'b_overdue_Mortgage_3', 'b_overdue_Car loan_3', 'p_is_several_a_day',
               'p_new_client_1', 'p_new_client_3', 'p_ins_ind']
for i in range(2, 22):
  categorical.append('FLAG_DOCUMENT_'+str(i))
real = list(set(train_cash.columns) - set(categorical))

In [ ]:
all_data = pd.concat([train_cash, test_cash])

In [ ]:
for c in real:
  all_data[c + '_bin'] = pd.qcut(all_data[c], 20, duplicates='drop').cat.add_categories("NAN").fillna("NAN")
train_cash = all_data[:train_cash.shape[0]]
test_cash = all_data[train_cash.shape[0]:]

In [ ]:
from sklearn.model_selection import train_test_split
train_cash, valid_cash, y_train_cash, y_valid_cash = train_test_split(train_cash, y_train_cash, test_size=0.2, random_state=42, stratify=(y_train_cash))
train_cash, count_cash, y_train_cash, y_count_cash = train_test_split(train_cash, y_train_cash, test_size=0.2, random_state=42, stratify=(y_train_cash))

In [ ]:
train_cash[categorical] = train_cash[categorical].fillna('NAN')
valid_cash[categorical] = valid_cash[categorical].fillna('NAN')
count_cash[categorical] = count_cash[categorical].fillna('NAN')
test_cash[categorical] = test_cash[categorical].fillna('NAN')

### Вычисление weight of evidence на отложенной выборке count

In [ ]:
woe_train_cash = train_cash.copy()
woe_valid_cash = valid_cash.copy()
woe_count_cash = count_cash.copy()
woe_test_cash = test_cash.copy()

In [ ]:
def calculate_woe_iv(data, column, target):
    lst = []
    for i in range(data[column].nunique()):
        val = list(data[column].unique())[i]
        lst.append({
            'Value': val,
            'All': data[data[column] == val].count()[column],
            'Good': data[(data[column] == val) & (target == 0)].count()[column],
            'Bad': data[(data[column] == val) & (target == 1)].count()[column]
        })

    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [ ]:
scorecard_cash = pd.DataFrame()

In [ ]:
res_cat = pd.DataFrame(index = categorical, columns = ['iv'])
for c in categorical:
  df, iv = calculate_woe_iv(count_cash, c, y_count_cash)
  df['feature'] = c
  for i in df.index:
    woe_train_cash.loc[woe_train_cash[c] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
    woe_valid_cash.loc[woe_valid_cash[c] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
    woe_test_cash.loc[woe_test_cash[c] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']

  res_cat.loc[c, 'iv'] = iv
  scorecard_cash = scorecard_cash.append(df[['feature', 'Value', 'WoE']])
    
res_cat[res_cat['iv'] >= 0.02]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

res_tree = pd.DataFrame(index = real, columns = ['iv'])

for c in real:
  d = pd.DataFrame(index = woe_count_cash.index, columns = ['woe'])
  df, iv = calculate_woe_iv(count_cash, c + '_bin', y_count_cash)
  df['feature'] = c
  for i in df.index:
    d.loc[woe_count_cash[c + '_bin'] == df.loc[i, 'Value'], 'woe'] = df.loc[i, 'WoE']

  clf = DecisionTreeClassifier(random_state = 42, max_depth = 2)
  clf.fit(d, y_count_cash)
  d['leaf'] = clf.apply(d)
  all_intervals = []
  for i in d['leaf'].unique():
    intervals = woe_count_cash[d['leaf'] == i][c+'_bin'].values.unique()
    woe_count_cash.loc[:, c+'_bin'] = woe_count_cash.loc[:, c+'_bin'].replace(intervals, i)
    woe_train_cash.loc[:, c+'_bin'] = woe_train_cash.loc[:, c+'_bin'].replace(intervals, i)
    woe_valid_cash.loc[:, c+'_bin'] = woe_valid_cash.loc[:, c+'_bin'].replace(intervals, i)
    woe_test_cash.loc[:, c+'_bin'] = woe_test_cash.loc[:, c+'_bin'].replace(intervals, i)
    all_intervals.append(str(intervals).split('\n')[0])

  df, iv = calculate_woe_iv(woe_count_cash, c + '_bin', y_count_cash)
  df['feature'] = c
  for i in df.index:
    woe_train_cash.loc[woe_train_cash[c + '_bin'] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
    woe_valid_cash.loc[woe_valid_cash[c + '_bin'] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
    woe_test_cash.loc[woe_test_cash[c + '_bin'] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
  df.loc[:, 'Value'] = df.loc[:, 'Value'].replace(d['leaf'].unique(), all_intervals)
  res_tree.loc[c, 'iv'] = iv
  scorecard_cash = scorecard_cash.append(df[['feature', 'Value', 'WoE']])

res_tree[res_tree['iv'] >= 0.02]

,iv
b_last_closed_Credit card,0.0233529
b_start_Consumer credit_avg,0.110112
p_Revolving loans_sum_app_avg,0.0218261
b_Credit card_sum_3,0.0413691
b_Credit card_avg,0.0302844
...,...
ENTRANCES_AVG,0.0269654
b_limit_min,0.0300389
b_Credit card_num_1,0.046701
b_active_all_dur_max,0.0509989


In [ ]:
iv_cash = pd.concat([res_cat, res_tree])

In [ ]:
scorecard_cash.to_csv('scorecard_cash.csv', index=True)

### Отбор признаков (по information value и корреляциям Спирмана)

In [ ]:
good_f = iv_cash[iv_cash['iv'] >= 0.02].index

In [ ]:
woe_train_cash1 = woe_train_cash[good_f]
woe_valid_cash1 = woe_valid_cash[good_f]
woe_test_cash1 = woe_test_cash[good_f]

In [ ]:
correlations_woe = woe_train_cash1.corr('spearman')
correlations_woe

,FLAG_EMP_PHONE,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,FLAG_DOCUMENT_6,b_last_closed_Credit card,b_start_Consumer credit_avg,p_Revolving loans_sum_app_avg,b_Credit card_sum_3,b_Credit card_avg,b_active_Consumer credit_dur_max,p_all_down_payment_avg_,b_Credit card_num,b_Car loan_sum,p_Consumer loans_down_payment_avg_,b_dur_avg_Consumer credit,FLOORSMAX_MEDI,b_late_num_all,b_early_num_all,p_prod_group_Cash Street_percent,p_sum,p_Consumer loans_sum_avg_,EXT_SOURCE_1,AMT_ANNUITY,b_Consumer credit_avg,b_active_Credit card_dur_avg,AMT_REQ_CREDIT_BUREAU_QRT,b_closed_days_avg_all,b_Consumer credit_sum_3,b_start_Credit card_avg,APARTMENTS_MEDI,b_Consumer credit_avg_1,b_Mortgage_avg,b_Car loan_avg,LIVINGAREA_MEDI,b_Consumer credit_num_1,YEARS_BUILD_AVG,b_debt_sum,b_now_overdue_percent_all,REGION_RATING_CLIENT_W_CITY,b_good_percent_avg,...,BASEMENTAREA_MEDI,p_cnt_avg,b_all_sum_3,APARTMENTS_MODE,p_Revolving loans_num,YEARS_BUILD_MODE,p_cnt_max,b_long_percent_avg,p_yield_low_normal,p_Consumer loans_sum_app_avg_3,b_closed_Credit card_num,b_last_closed_Car loan,b_all_num_1,b_all_avg,b_last_closed_all,DAYS_BIRTH,b_Mortgage_num,YEARS_BEGINEXPLUATATION_AVG,b_Credit card_mean,AMT_REQ_CREDIT_BUREAU_MON,NONLIVINGAREA_AVG,b_active_Credit card_num,b_debt_num,b_all_sum_1,AMT_GOODS_PRICE,AMT_CREDIT,p_Consumer loans_high_percent,b_debt_avg_Credit card,NONLIVINGAREA_MODE,ENTRANCES_MODE,LIVINGAREA_MODE,b_Credit card_num_3,p_all_num_3,b_Mortgage_mean,b_all_num,ENTRANCES_AVG,b_limit_min,b_Credit card_num_1,b_active_all_dur_max,p_evening_percent
FLAG_EMP_PHONE,1.000000,0.093636,0.261971,0.615325,0.031271,0.034352,0.039073,0.030811,0.005735,0.018287,-0.045206,0.041183,-0.040700,-0.037482,0.037297,-0.012412,-0.000163,-0.001424,0.013696,0.034810,-0.005741,0.027197,0.047414,0.026660,0.019141,-0.004369,0.025020,0.026209,0.005001,-0.009207,-0.006643,-0.044051,-0.040614,-0.012928,-0.011577,-0.007084,0.041115,-0.009529,-0.013559,0.005133,...,-0.008933,-0.070840,0.004222,-0.008932,0.029683,-0.007753,-0.011416,-0.008799,0.067884,0.013014,-0.015048,0.031271,0.004234,-0.046116,0.031271,0.562456,-0.012799,-0.009122,0.025575,-0.018059,-0.011263,0.037677,0.023976,0.020066,-0.003360,0.003162,-0.010999,0.035449,-0.010890,-0.005861,-0.012039,0.037917,-0.031626,-0.044032,-0.012912,-0.006853,0.010915,0.003540,0.044890,-0.101452
REG_CITY_NOT_LIVE_CITY,0.093636,1.000000,0.437358,0.059096,0.015454,0.083277,0.031103,0.028529,0.021427,0.007815,0.006752,0.011523,0.026433,0.009316,0.064248,0.159357,0.043951,0.046592,-0.008791,0.037691,-0.001279,0.068178,0.011153,0.036826,0.026363,0.032491,0.079888,0.011117,0.054021,0.152494,0.020291,0.024116,0.026579,0.152266,0.023960,0.108837,0.015544,0.028234,0.022552,0.027331,...,0.129704,-0.020706,0.015667,0.149333,0.018090,0.110345,-0.007938,0.029578,0.039317,0.022327,0.030418,0.015454,0.026708,0.016653,0.015454,0.170125,0.029678,0.141021,0.023117,0.032373,0.141636,0.016966,0.012088,0.028191,0.037093,0.026740,0.027998,0.030395,0.141744,0.149586,0.148418,0.027319,-0.001231,0.024073,0.026699,0.149427,0.037134,0.030844,0.022072,-0.019155
REG_CITY_NOT_WORK_CITY,0.261971,0.437358,1.000000,0.165817,0.011782,0.067379,0.030672,0.013361,0.007173,0.018725,-0.007222,-0.004133,0.013210,-0.003939,0.051817,0.268478,0.040384,0.039747,-0.001495,0.046241,0.011569,0.085237,0.029617,0.035358,0.018845,0.025844,0.069045,0.014764,0.050550,0.256354,0.011254,0.022188,0.013078,0.259986,0.015864,0.186790,0.025294,0.023961,0.005601,0.024540,...,0.218223,-0.023696,0.016492,0.251185,0.022678,0.188748,0.001293,0.023766,0.052918,0.032821,0.029703,0.011782,0.019066,0.009631,0.011782,0.225919,0.023740,0.234396,0.009594,0.029162,0.239724,0.007533,0.015698,0.020082,0.038747,0.027016,0.025443,0.023763,0.241043,0.248358,0.251474,0.014915,-0.007143,0.022141,0.019604,0.249164,0.026239,0.023777,0.027834,-0.007429
FLAG_DOCUMENT_6,0.615325,0.059096,0.165817,1.000000,0.022153,0.016839,0.052297,0.024176,0.009595,0.012729,-0.017892,0.030787,-0.025387,-0.013768,0.023186,-0.004970,-0.

In [ ]:
features = correlations_woe.columns
corr = []
for i in range(len(features)-1):
  for j in range(i+1, len(features)):
    if abs(correlations_woe.iloc[i, j]) >= 0.8:
       if iv_cash.loc[features[i], 'iv'] > iv_cash.loc[features[j], 'iv']:
         corr.append(features[j])
       else:
         corr.append(features[i])
woe_train_cash1 = woe_train_cash1.drop(columns=corr)
woe_valid_cash1 = woe_valid_cash1.drop(columns=corr)
woe_test_cash1 = woe_test_cash1.drop(columns=corr)

In [ ]:
from sklearn.metrics import roc_auc_score

def Gini(y_true, y_pred):
  return (2*roc_auc_score(y_true.values, y_pred) - 1)

### Обучение модели

In [ ]:
from sklearn.linear_model import LogisticRegression
best_g = 0
best_c = 0
for c in [0.1, 0.3, 0.5, 0.7, 1]:
  clf = LogisticRegression(random_state=42, C=c).fit(woe_train_cash1, y_train_cash)
  pred = clf.predict_proba(woe_valid_cash1)
  er = Gini(y_valid_cash, pred[:, 1])
  if best_g == 0 or er > best_g:
    best_g = er
    best_c = c
print('best Gini =', best_g, '\nbest C =', best_c)

best Gini = 0.5045837514370708 
best C = 1


In [ ]:
woe_train_all_cash = pd.concat([woe_train_cash1, woe_valid_cash1])
y_train_all_cash = pd.concat([y_train_cash, y_valid_cash])
clf = LogisticRegression(random_state=42, C=best_c).fit(woe_train_all_cash, y_train_all_cash)
pred = clf.predict_proba(woe_test_cash1)
print('test Gini =', Gini(y_test_cash, pred[:, 1]))
print('test AUC_ROC =', roc_auc_score(y_test_cash.values, pred[:, 1]))

test Gini = 0.4832941279537375
test AUC_ROC = 0.7416470639768687


### Отбор признаков forward и backward selection

In [ ]:
features_cash = []
prev_gini = 0
while len(features_cash) < 30:
    f = 0
    best_change = -1
    for i in range(woe_train_cash1.shape[1]):
        if (~(i in features_cash)):
            clf = LogisticRegression(random_state=42, C=best_c)
            features_cash.append(i)
            clf.fit(woe_train_cash1.iloc[:, features_cash], y_train_cash)
            pred = clf.predict_proba(woe_valid_cash1.iloc[:, features_cash])
            gini = Gini(y_valid_cash, pred[:, 1])
            features_cash.pop()
            if ((best_change == -1) | ((gini - prev_gini) > best_change)):
                best_change = gini - prev_gini
                f = i
    features_cash.append(f)

In [ ]:
woe_train_cash1 = woe_train_cash1.iloc[:, features_cash]
woe_valid_cash = woe_valid_cash1.iloc[:, features_cash]
woe_test_cash1 = woe_test_cash1.iloc[:, features_cash]
woe_train_cash1.to_csv('woe_train_cash1.csv', index=True)
woe_valid_cash1.to_csv('woe_valid_cash1.csv', index=True)
woe_test_cash1.to_csv('woe_test_cash1.csv', index=True)

In [ ]:
clf = LogisticRegression(random_state=42, C=best_c)
clf.fit(woe_train_cash1, y_train_cash)
pred = clf.predict_proba(woe_valid_cash1)
prev_gini = Gini(y_valid_cash, pred[:, 1])
features_cash = np.arange(0, woe_train_cash1.shape[1])
while len(features_cash) > 15:
  f = 0
  best_change = -1
  for i in range(len(features_cash)):
    clf = LogisticRegression(random_state=42, C=best_c)
    feat = np.delete(features_cash, i)
    clf.fit(woe_train_cash1.iloc[:, feat], y_train_cash)
    pred = clf.predict_proba(woe_valid_cash1.iloc[:, feat])
    gini = Gini(y_valid_cash, pred[:, 1])
    if ((gini - prev_gini) > best_change):
      best_change = gini - prev_gini
      f = i
  features_cash = np.delete(features_cash, f)
  prev_gini += best_change
len(features_cash)

### Обучение финальной модели для cash loans

In [ ]:
best_g = 0
best_c_cash = 0
for c in [0.1, 0.3, 0.5, 0.7, 1]:
  clf = LogisticRegression(random_state=42, C=c).fit(woe_train_cash1.iloc[:, features_cash], y_train_cash)
  pred = clf.predict_proba(woe_valid_cash1.iloc[:, features_cash])
  er = Gini(y_valid_cash, pred[:, 1])
  if best_g == 0 or er > best_g:
    best_g = er
    best_c_cash = c
print('best Gini =', best_g, '\nbest C =', best_c_cash)

In [ ]:
woe_train_all_cash = pd.concat([woe_train_cash1, woe_valid_cash1])
y_train_all_cash = pd.concat([y_train_cash, y_valid_cash])
clf = LogisticRegression(random_state=42, C=best_c_cash).fit(woe_train_all_cash.iloc[:, features_cash], y_train_all_cash)
pred = clf.predict_proba(woe_test_cash1.iloc[:, features_cash])
print('test Gini =', Gini(y_test_cash, pred[:, 1]))
print('test AUC_ROC =', roc_auc_score(y_test_cash.values, pred[:, 1]))

In [ ]:
woe_train_all_cash.iloc[:, features_cash].to_csv('woe_all_train_cash_final.csv', index=True)
woe_test_cash1.iloc[:, features_cash].to_csv('woe_test_cash_final.csv', index=True)

#Revolving loans

In [ ]:
all_data = pd.concat([train_rev, test_rev])

In [ ]:
for c in real:
  all_data[c + '_bin'] = pd.qcut(all_data[c], 20, duplicates='drop').cat.add_categories("0-0").fillna("0-0")
train_rev = all_data[:train_rev.shape[0]]
test_rev = all_data[train_rev.shape[0]:]

In [ ]:
from sklearn.model_selection import train_test_split
train_rev, valid_rev, y_train_rev, y_valid_rev = train_test_split(train_rev, y_train_rev, test_size=0.2, random_state=42, stratify=(y_train_rev))

In [ ]:
train_rev[categorical] = train_rev[categorical].fillna('NAN')
valid_rev[categorical] = valid_rev[categorical].fillna('NAN')
test_rev[categorical] = test_rev[categorical].fillna('NAN')

In [ ]:
scorecard_rev = pd.DataFrame()

In [ ]:
woe_train_rev = train_rev.copy()
woe_valid_rev = valid_rev.copy()
woe_test_rev = test_rev.copy()

In [ ]:
res_cat = pd.DataFrame(index = categorical, columns = ['iv'])
for c in categorical:
  df, iv = calculate_woe_iv(train_rev, c, y_train_rev)
  df['feature'] = c
  for i in df.index:
    woe_train_rev.loc[woe_train_rev[c] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
    woe_valid_rev.loc[woe_valid_rev[c] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
    woe_test_rev.loc[woe_test_rev[c] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']

  res_cat.loc[c, 'iv'] = iv
  scorecard_rev = scorecard_rev.append(df[['feature', 'Value', 'WoE']])
    
res_cat[res_cat['iv'] >= 0.02]

,iv
NAME_INCOME_TYPE,0.062273
NAME_EDUCATION_TYPE,0.108189
NAME_FAMILY_STATUS,0.034721
NAME_HOUSING_TYPE,0.0350167
OCCUPATION_TYPE,0.127442
ORGANIZATION_TYPE,0.0993023
FONDKAPREMONT_MODE,0.0352369
HOUSETYPE_MODE,0.0363533
WALLSMATERIAL_MODE,0.0688655
EMERGENCYSTATE_MODE,0.0441584


In [ ]:
from sklearn.tree import DecisionTreeClassifier

res_tree = pd.DataFrame(index = real, columns = ['iv'])

for c in real:
  d = pd.DataFrame(index = woe_train_rev.index, columns = ['woe'])
  df, iv = calculate_woe_iv(train_rev, c + '_bin', y_train_rev)
  df['feature'] = c
  for i in df.index:
    d.loc[woe_train_rev[c + '_bin'] == df.loc[i, 'Value'], 'woe'] = df.loc[i, 'WoE']

  clf = DecisionTreeClassifier(random_state = 42, max_depth = 2)
  clf.fit(d, y_train_rev)
  d['leaf'] = clf.apply(d)
  all_intervals = []
  for i in d['leaf'].unique():
    intervals = woe_train_rev[d['leaf'] == i][c+'_bin'].values.unique()
    woe_train_rev.loc[:, c+'_bin'] = woe_train_rev.loc[:, c+'_bin'].replace(intervals, i)
    woe_valid_rev.loc[:, c+'_bin'] = woe_valid_rev.loc[:, c+'_bin'].replace(intervals, i)
    woe_test_rev.loc[:, c+'_bin'] = woe_test_rev.loc[:, c+'_bin'].replace(intervals, i)
    all_intervals.append(str(intervals).split('\n')[0])

  df, iv = calculate_woe_iv(woe_train_rev, c + '_bin', y_train_rev)
  df['feature'] = c
  for i in df.index:
    woe_train_rev.loc[woe_train_rev[c + '_bin'] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
    woe_valid_rev.loc[woe_valid_rev[c + '_bin'] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
    woe_test_rev.loc[woe_test_rev[c + '_bin'] == df.loc[i, 'Value'], c] = df.loc[i, 'WoE']
  df.loc[:, 'Value'] = df.loc[:, 'Value'].replace(d['leaf'].unique(), all_intervals)
  res_tree.loc[c, 'iv'] = iv
  scorecard_rev = scorecard_rev.append(df[['feature', 'Value', 'WoE']])

res_tree[res_tree['iv'] >= 0.02]

,iv
b_last_closed_Credit card,0.0360395
b_start_Consumer credit_avg,0.153994
b_Credit card_sum_3,0.0284058
b_Credit card_avg,0.0473227
b_active_Consumer credit_dur_max,0.0697788
...,...
b_all_num,0.0302053
ENTRANCES_AVG,0.0513033
b_limit_min,0.0270723
b_Credit card_num_1,0.0346794


In [ ]:
iv_rev = pd.concat([res_cat, res_tree])
good_f = iv_rev[iv_rev['iv'] >= 0.02].index

In [ ]:
scorecard_rev.to_csv('scorecard_rev.csv', index=True)
iv_cash.to_csv('iv_cash.csv', index=True)
iv_rev.to_csv('iv_rev.csv', index=True)

In [ ]:
woe_train_rev1 = woe_train_rev[good_f]
woe_valid_rev1 = woe_valid_rev[good_f]
woe_test_rev1 = woe_test_rev[good_f]

In [ ]:
correlations_woe = woe_train_rev1.corr('spearman')
correlations_woe

,FLAG_WORK_PHONE,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,FLAG_DOCUMENT_3,b_last_closed_Credit card,b_start_Consumer credit_avg,b_Credit card_sum_3,b_Credit card_avg,b_active_Consumer credit_dur_max,p_type_suite_Family_percent,p_all_down_payment_avg_,p_prod_group_POS household_percent,p_Consumer loans_down_payment_avg_,b_dur_avg_Consumer credit,LANDAREA_AVG,FLOORSMAX_MEDI,b_early_num_all,p_sum,p_Consumer loans_sum_avg_,EXT_SOURCE_1,AMT_ANNUITY,b_Consumer credit_avg,b_active_Credit card_dur_avg,b_closed_days_avg_all,b_active_Mortgage_dur_max,b_Consumer credit_sum_3,b_start_Credit card_avg,APARTMENTS_MEDI,b_Consumer credit_avg_1,AMT_INCOME_TOTAL,b_Mortgage_avg,LIVINGAREA_MEDI,b_Consumer credit_num_1,YEARS_BUILD_AVG,b_debt_sum,LANDAREA_MODE,OBS_60_CNT_SOCIAL_CIRCLE,LIVINGAPARTMENTS_MEDI,p_Consumer loans_sum_avg_1,b_all_num_3,...,p_all_low_percent,EXT_SOURCE_2,BASEMENTAREA_MEDI,p_cnt_avg,b_all_sum_3,APARTMENTS_MODE,FLOORSMIN_AVG,YEARS_BUILD_MODE,p_all_sum_app_avg_1,LIVINGAPARTMENTS_MODE,OBS_30_CNT_SOCIAL_CIRCLE,p_yield_low_normal,p_Consumer loans_sum_app_avg_3,b_last_closed_Car loan,b_all_num_1,b_all_avg,NONLIVINGAPARTMENTS_MODE,b_last_closed_all,DAYS_BIRTH,YEARS_BEGINEXPLUATATION_AVG,b_Credit card_mean,AMT_REQ_CREDIT_BUREAU_MON,NONLIVINGAREA_AVG,b_debt_num,b_all_sum_1,AMT_GOODS_PRICE,AMT_CREDIT,b_debt_avg_Credit card,p_prod_group_POS mobile_percent,DEF_30_CNT_SOCIAL_CIRCLE,NONLIVINGAREA_MODE,ENTRANCES_MODE,LIVINGAREA_MODE,b_Credit card_num_3,b_Mortgage_mean,b_all_num,ENTRANCES_AVG,b_limit_min,b_Credit card_num_1,b_active_all_dur_max
FLAG_WORK_PHONE,1.000000,0.072000,0.121056,0.058850,-0.009527,0.059059,0.028143,0.035070,0.028922,-0.029682,-0.033360,-0.033493,-0.020240,0.043232,0.038538,0.058399,0.061336,0.054192,0.037167,0.011888,0.066054,0.037135,0.050435,0.039169,0.063160,0.037945,0.029056,0.046341,0.033201,0.072580,0.060619,0.040850,0.051687,0.039738,0.009626,0.037572,0.047212,0.043233,0.027202,0.037087,...,-0.073414,0.026574,0.036979,-0.000041,0.042013,0.036486,0.044731,0.038597,-0.011236,0.040701,0.044357,-0.000391,0.044113,-0.009527,0.040393,0.057775,0.041021,-0.009527,0.099636,0.037646,0.035194,0.075464,0.041948,0.036112,0.018251,0.113145,0.066054,0.012657,0.042094,0.067487,0.035054,0.047084,0.043141,0.025421,0.060852,0.060986,0.044151,0.056069,0.052718,0.016068
REG_CITY_NOT_LIVE_CITY,0.072000,1.000000,0.484076,0.010096,0.013926,0.083048,0.013302,0.039259,0.003724,0.016789,0.010299,-0.010010,0.005995,0.054502,0.151792,0.171042,0.043004,0.049006,0.008334,0.078954,0.102767,0.009886,0.032279,0.091532,0.022691,0.006916,0.069338,0.183571,0.011547,0.025012,0.021491,0.179702,0.022394,0.129350,0.007197,0.137611,0.008721,0.106956,0.028559,0.012396,...,-0.004650,0.062952,0.143597,0.011683,0.011243,0.158738,0.134488,0.131598,0.038806,0.114828,-0.000037,0.027670,0.037685,0.013926,0.026400,0.032352,0.133031,0.013926,0.174752,0.172723,0.031173,0.028461,0.170152,0.007794,0.019643,0.100632,0.102767,0.019652,0.050223,0.023210,0.153824,0.178999,0.164821,0.027702,0.021212,0.021422,0.179093,0.025875,0.026127,0.014740
REG_CITY_NOT_WORK_CITY,0.121056,0.484076,1.000000,0.008389,0.006975,0.067396,0.012893,0.040159,0.022860,0.024369,0.009060,0.004226,0.007451,0.024617,0.244736,0.275195,0.035842,0.058258,0.030916,0.094471,0.114345,0.028548,0.026400,0.067828,0.022935,0.026386,0.060466,0.285437,0.024021,0.029147,0.025928,0.271390,0.018154,0.194551,0.014569,0.225182,0.013865,0.175614,0.029550,0.001129,...,-0.004061,0.094699,0.230415,0.018754,0.013422,0.249174,0.212492,0.198921,0.043793,0.184507,0.013415,0.051642,0.050532,0.006975,0.012761,0.034603,0.207176,0.006975,0.192634,0.262190,0.038705,0.031279,0.266013,0.016995,0.015300,0.110612,0.114345,0.008509,0.061790,0.020199,0.233213,0.275308,0.260288,-0.001661,0.027137,0.020121,0.276305,0.020663,0.015256,0.015190
FLAG_DOCUMENT_3,0.058850,0.010096,0.008389,1.000000,0.021349,0.020983,0.007035,0.005871,0.031420,-0.070032,-0.061723,-0.077269,-0.047823,0.004920,-0.000648,-0.000824,0.023977,-0.007760,-0.0

In [ ]:
features = correlations_woe.columns
corr = []
for i in range(len(features)-1):
  for j in range(i+1, len(features)):
    if abs(correlations_woe.iloc[i, j]) >= 0.8:
       if iv_rev.loc[features[i], 'iv'] > iv_rev.loc[features[j], 'iv']:
         corr.append(features[j])
       else:
         corr.append(features[i])
woe_train_rev1 = woe_train_rev1.drop(columns=corr)
woe_valid_rev1 = woe_valid_rev1.drop(columns=corr)
woe_test_rev1 = woe_test_rev1.drop(columns=corr)

In [ ]:
from sklearn.linear_model import LogisticRegression
best_g = 0
best_c = 0
for c in [0.1, 0.3, 0.5, 0.7, 1]:
  clf = LogisticRegression(random_state=42, C=c).fit(woe_train_rev1, y_train_rev)
  pred = clf.predict_proba(woe_valid_rev1)
  er = Gini(y_valid_rev, pred[:, 1])
  if best_g == 0 or er > best_g:
    best_g = er
    best_c = c
print('best Gini =', best_g, '\nbest C =', best_c)

best Gini = 0.4398526845343136 
best C = 0.1


In [ ]:
clf = LogisticRegression(random_state=42, C=best_c).fit(woe_train_rev, y_train_rev)
pred = clf.predict_proba(woe_test_rev)
print('test Gini =', Gini(y_test_rev, pred[:, 1]))
print('test AUC_ROC =', roc_auc_score(y_test_rev.values, pred[:, 1]))

test Gini = 0.45711226910278047
test AUC_ROC = 0.7285561345513902


In [ ]:
features_rev = []
prev_gini = 0
while len(features_rev) < 30:
    f = 0
    best_change = -1
    for i in range(woe_train_rev1.shape[1]):
        if (~(i in features_rev)):
            clf = LogisticRegression(random_state=42, C=best_c)
            features_rev.append(i)
            clf.fit(woe_train_rev1.iloc[:, features_rev], y_train_rev)
            pred = clf.predict_proba(woe_valid_rev1.iloc[:, features_rev])
            gini = Gini(y_valid_rev, pred[:, 1])
            features_rev.pop()
            if ((best_change == -1) | ((gini - prev_gini) > best_change)):
                best_change = gini - prev_gini
                f = i
    features_rev.append(f)

In [ ]:
woe_train_rev1 = woe_train_rev1.iloc[:, features_rev]
woe_valid_rev = woe_valid_rev1.iloc[:, features_rev]
woe_test_rev1 = woe_test_rev1.iloc[:, features_rev]
woe_train_rev1.to_csv('woe_train_rev1.csv', index=True)
woe_valid_rev1.to_csv('woe_valid_rev1.csv', index=True)
woe_test_rev1.to_csv('woe_test_rev1.csv', index=True)

In [ ]:
clf = LogisticRegression(random_state=42, C=best_c)
clf.fit(woe_train_rev1, y_train_rev)
pred = clf.predict_proba(woe_valid_rev1)
prev_gini = Gini(y_valid_rev, pred[:, 1])
features_rev = np.arange(0, woe_train_rev1.shape[1])
while len(features_rev) > 15:
  f = 0
  best_change = -1
  for i in range(len(features_rev)):
    clf = LogisticRegression(random_state=42, C=best_c)
    feat = np.delete(features_rev, i)
    clf.fit(woe_train_rev1.iloc[:, feat], y_train_rev)
    pred = clf.predict_proba(woe_valid_rev1.iloc[:, feat])
    gini = Gini(y_valid_rev, pred[:, 1])
    if ((gini - prev_gini) > best_change):
      best_change = gini - prev_gini
      f = i
  features_rev = np.delete(features_rev, f)
  prev_gini += best_change
len(features_rev)

In [ ]:
best_g = 0
best_c_rev = 0
for c in [0.1, 0.3, 0.5, 0.7, 1]:
  clf = LogisticRegression(random_state=42, C=c).fit(woe_train_rev1, y_train_rev)
  pred = clf.predict_proba(woe_valid_rev1)
  er = Gini(y_valid_rev, pred[:, 1])
  if best_g == 0 or er > best_g:
    best_g = er
    best_c_rev = c
print('best Gini =', best_g, '\nbest C =', best_c_rev)

In [ ]:
woe_train_all_rev = pd.concat([woe_train_rev1, woe_valid_rev1])
y_train_all_rev = pd.concat([y_train_rev, y_valid_rev])
clf = LogisticRegression(random_state=42, C=best_c_rev).fit(woe_train_all_rev.iloc[:, features_rev], y_train_all_rev)
pred = clf.predict_proba(woe_test_rev1.iloc[:, features_rev])
print('test Gini =', Gini(y_test_rev, pred[:, 1]))
print('test AUC_ROC =', roc_auc_score(y_test_rev.values, pred[:, 1]))

In [ ]:
woe_train_all_rev.iloc[:, features_rev].to_csv('woe_all_train_rev_final.csv', index=True)
woe_test_rev1.iloc[:, features_rev].to_csv('woe_test_rev_final.csv', index=True)

##Результат на нашем тесте

In [ ]:
clf_cash = LogisticRegression(random_state=42, C=best_c_cash).fit(woe_train_all_cash.iloc[:, features_cash], y_train_all_cash)
pred_cash = pd.DataFrame(clf_cash.predict_proba(woe_test_cash1.iloc[:, features_cash]), index=woe_test_cash1.index)

clf_rev = LogisticRegression(random_state=42, C=best_c_rev).fit(woe_train_all_rev.iloc[:, features_rev], y_train_all_rev)
pred_rev = pd.DataFrame(clf_rev.predict_proba(woe_test_rev1.iloc[:, features_rev]), index=woe_test_rev1.index)

pred_test = pd.concat([pred_cash, pred_rev])
y_test = pd.concat([y_test_cash, y_test_rev])

print('test Gini =', Gini(y_test, pred_test.iloc[:,1]))
print('test AUC_ROC =', roc_auc_score(y_test.values, pred_test.iloc[:,1]))

## Построение скоркарты

In [1]:
import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.metrics import roc_auc_score

def Gini(y_true, y_pred):
  return (2*roc_auc_score(y_true.values, y_pred) - 1)

In [ ]:
woe_test_cash = pd.read_csv('/content/drive/My Drive/woe_test_cash1.csv').set_index('SK_ID_CURR')
woe_train_cash = pd.read_csv('/content/drive/My Drive/woe_train_cash1.csv').set_index('SK_ID_CURR')
woe_test_rev = pd.read_csv('/content/drive/My Drive/woe_test_rev1.csv').set_index('SK_ID_CURR')
woe_train_rev = pd.read_csv('/content/drive/My Drive/woe_train_rev1.csv').set_index('SK_ID_CURR')

In [ ]:
y_test_cash = pd.read_csv('/content/drive/My Drive/y_test_cash.csv').set_index('SK_ID_CURR')
y_train_cash = pd.read_csv('/content/drive/My Drive/y_train_cash.csv').set_index('SK_ID_CURR')
y_test_rev = pd.read_csv('/content/drive/My Drive/y_test_rev.csv').set_index('SK_ID_CURR')
y_train_rev = pd.read_csv('/content/drive/My Drive/y_train_rev.csv').set_index('SK_ID_CURR')

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_rev = LogisticRegression(random_state=42, C=0.1).fit(woe_train_rev, y_train_rev)
pred_rev = clf_rev.predict_proba(woe_test_rev)
print('test Gini =', Gini(y_test_rev, pred_rev[:, 1]))
print('test AUC_ROC =', roc_auc_score(y_test_rev.values, pred_rev[:, 1]))

clf_cash = LogisticRegression(random_state=42, C=0.1).fit(woe_train_cash, y_train_cash)
pred_cash = clf_cash.predict_proba(woe_test_cash)
print('test Gini =', Gini(y_test_cash, pred_cash[:, 1]))
print('test AUC_ROC =', roc_auc_score(y_test_cash.values, pred_cash[:, 1]))

test Gini = 0.48576706507755474
test AUC_ROC = 0.7428835325387774
test Gini = 0.47251207062438483
test AUC_ROC = 0.7362560353121924


In [ ]:
coef_cash = pd.DataFrame(columns = ['feature', 'coef'])
coef_cash['feature'] = woe_train_cash.columns.values
coef_cash['coef'] = clf_cash.coef_[0]

coef_rev = pd.DataFrame(columns = ['feature', 'coef'])
coef_rev['feature'] = woe_train_rev.columns.values
coef_rev['coef'] = clf_rev.coef_[0]

In [ ]:
coef_cash = coef_cash[coef_cash['coef'] < -0.3]
clf_cash = LogisticRegression(random_state=42, C=0.1).fit(woe_train_cash[coef_cash['feature']], y_train_cash)
pred_cash = clf_cash.predict_proba(woe_test_cash[coef_cash['feature']])
print('test Gini =', Gini(y_test_cash, pred_cash[:, 1]))
print('test AUC_ROC =', roc_auc_score(y_test_cash.values, pred_cash[:, 1]))

test Gini = 0.4604392095347627
test AUC_ROC = 0.7302196047673813


In [ ]:
coef_rev = coef_rev[coef_rev['coef'] < -0.3]
clf_rev = LogisticRegression(random_state=42, C=0.1).fit(woe_train_rev[coef_rev['feature']], y_train_rev)
pred_rev = clf_rev.predict_proba(woe_test_rev[coef_rev['feature']])
print('test Gini =', Gini(y_test_rev, pred_rev[:, 1]))
print('test AUC_ROC =', roc_auc_score(y_test_rev.values, pred_rev[:, 1]))

test Gini = 0.4804451459479342
test AUC_ROC = 0.7402225729739671


In [ ]:
pred_test = pd.concat([pd.DataFrame(pred_cash, index=woe_test_cash.index), pd.DataFrame(pred_rev, index=woe_test_rev.index)])
y_test = pd.concat([y_test_cash, y_test_rev])

print('test Gini =', Gini(y_test, pred_test.iloc[:,1]))
print('test AUC_ROC =', roc_auc_score(y_test.values, pred_test.iloc[:,1]))

test Gini = 0.4652468071881404
test AUC_ROC = 0.7326234035940702


In [ ]:
scorecard_cash = pd.read_csv('/content/drive/My Drive/scorecard_cash.csv').set_index('Unnamed: 0').rename_axis(None, axis=1).rename_axis('', axis=0)
scorecard_rev = pd.read_csv('/content/drive/My Drive/scorecard_rev.csv').set_index('Unnamed: 0').rename_axis(None, axis=1).rename_axis('', axis=0)
iv_cash = pd.read_csv('/content/drive/My Drive/iv_cash.csv')
iv_rev = pd.read_csv('/content/drive/My Drive/iv_rev.csv')

In [ ]:
scorecard_cash['iv'] = 0
for f in scorecard_cash['feature'].unique():
  scorecard_cash.loc[scorecard_cash['feature'] == f, 'iv'] = iv_cash[iv_cash['Unnamed: 0'] == f]['iv'].values[0]

In [ ]:
scorecard_rev['iv'] = 0
for f in scorecard_rev['feature'].unique():
  scorecard_rev.loc[scorecard_rev['feature'] == f, 'iv'] = iv_rev[iv_rev['Unnamed: 0'] == f]['iv'].values[0]

In [ ]:
name_features_cash = list(coef_cash['feature'])
name_features_rev = list(coef_rev['feature'])
scorecard_cash['feature'] = np.vectorize(lambda x: x if x in name_features_cash else '')(scorecard_cash['feature'])
scorecard_cash = scorecard_cash[scorecard_cash['feature'] != '']
scorecard_rev['feature'] = np.vectorize(lambda x: x if x in name_features_rev else '')(scorecard_rev['feature'])
scorecard_rev = scorecard_rev[scorecard_rev['feature'] != '']

In [ ]:
scorecard_cash = scorecard_cash.sort_values(by='iv', ascending=False)
scorecard_rev = scorecard_rev.sort_values(by='iv', ascending=False)
scorecard_cash.index = np.arange(0, scorecard_cash.shape[0])
scorecard_rev.index = np.arange(0, scorecard_rev.shape[0])

In [ ]:
R = 40 / np.log(2)
A = 600 - R * np.log(72)
scorecard_cash_final['score'] = 0
for f in scorecard_cash_final['feature'].unique():
  scorecard_cash_final.loc[scorecard_cash_final['feature'] == f, 'score'] = (scorecard_cash_final[scorecard_cash_final['feature'] == f]['WoE']*coef_cash[coef_cash['feature'] == f]['coef'].values[0] - clf_cash.intercept_[0]/17)*R + A/17
scorecard_rev_final['score'] = 0
for f in scorecard_rev_final['feature'].unique():
  scorecard_rev_final.loc[scorecard_rev_final['feature'] == f, 'score'] = (scorecard_rev_final[scorecard_rev_final['feature'] == f]['WoE']*coef_rev[coef_rev['feature'] == f]['coef'].values[0] - clf_rev.intercept_[0]/16)*R + A/16

In [14]:
scorecard_cash_final = pd.read_csv('/content/drive/My Drive/scorecard_cash_final.csv').set_index('Unnamed: 0').rename_axis(None, axis=1).rename_axis('', axis=0)
scorecard_rev_final = pd.read_csv('/content/drive/My Drive/scorecard_rev_final.csv').set_index('Unnamed: 0').rename_axis(None, axis=1).rename_axis('', axis=0)

#### Парсинг строкового представление интервалов в pd.Interval

In [3]:
res_res = []
for i in range(scorecard_cash_final.shape[0]):
  s = scorecard_cash_final.loc[i, 'Value']
  j = 0
  if s[:9] != '[Interval':
    res = [[s]]
    res_res.append(res)
    continue
  res = []
  while j < len(s):
    while j < len(s) and s[j] != '(':
      if s[j] == 'N':
        res.append(['NAN'])
        j += 3
      else:
        j += 1
    first = ''
    j += 1
    while j < len(s) and s[j] != ',':
      first += s[j]
      j += 1
    j += 1
    second = ''
    while j < len(s) and s[j] != ',':
      second += s[j]
      j += 1
    if first != '' and second != '':
      res.append(pd.Interval(left=float(first), right=float(second), closed='right'))  
  res_res.append(res)

    


In [ ]:
scorecard_cash_final.loc[:, 'Value'] = res_res

In [24]:
res_res = []
for i in range(scorecard_rev_final.shape[0]):
  s = scorecard_rev_final.loc[i, 'Value']
  j = 0
  if s[:9] != '[Interval':
    res = [[s]]
    res_res.append(res)
    continue
  res = []
  while j < len(s):
    while j < len(s) and s[j] != '(':
      if s[j] == 'N':
        res.append(['NAN'])
        j += 3
      else:
        j += 1
    first = ''
    j += 1
    while j < len(s) and s[j] != ',':
      first += s[j]
      j += 1
    j += 1
    second = ''
    while j < len(s) and s[j] != ',':
      second += s[j]
      j += 1
    if first != '' and second != '':
      res.append('('+str(first)+', '+str(second)+']')  
  res_res.append(res)

    


In [25]:
scorecard_rev_final.loc[:, 'Value'] = res_res

In [ ]:
scorecard_cash_final.loc[:, 'WoE'] = np.round(scorecard_cash_final.loc[:, 'WoE'], decimals=3)
scorecard_cash_final.loc[:, 'iv'] = np.round(scorecard_cash_final.loc[:, 'iv'], decimals=3)
scorecard_cash_final.loc[:, 'score'] = np.round(scorecard_cash_final.loc[:, 'score'], decimals=3)
scorecard_cash_final.to_csv('cash_beautiful.csv')

In [26]:
scorecard_rev_final.loc[:, 'WoE'] = np.round(scorecard_rev_final.loc[:, 'WoE'], decimals=3)
scorecard_rev_final.loc[:, 'iv'] = np.round(scorecard_rev_final.loc[:, 'iv'], decimals=3)
scorecard_rev_final.loc[:, 'score'] = np.round(scorecard_rev_final.loc[:, 'score'], decimals=3)
scorecard_rev_final.to_csv('rev_beautiful.csv')

In [ ]:
from sklearn.model_selection import train_test_split
a, valid_cash, b, y_valid_cash = train_test_split(train_cash, y_train_cash, test_size=0.2, random_state=11, stratify=y_train_cash)
b, valid_rev, b, y_valid_rev = train_test_split(train_rev, y_train_rev, test_size=0.2, random_state=11, stratify=y_train_rev)

In [ ]:
test_cash = test_cash[scorecard_cash_final['feature'].unique()]
test_rev = test_rev[scorecard_rev_final['feature'].unique()]

In [ ]:
test_cash = test_cash.fillna('NAN')
test_rev = test_rev.fillna('NAN')

In [ ]:
valid_cash = valid_cash[scorecard_cash_final['feature'].unique()]
valid_rev = valid_rev[scorecard_rev_final['feature'].unique()]
valid_cash = valid_cash.fillna('NAN')
valid_rev = valid_rev.fillna('NAN')

In [ ]:
score_cash = pd.Series(0, index=test_cash.index)
score_rev = pd.Series(0, index=test_rev.index)

In [ ]:
score_cash_valid = pd.Series(0, index=valid_cash.index)
score_rev_valid = pd.Series(0, index=valid_rev.index)

In [ ]:
for i in test_cash.index:
  for f in scorecard_cash_final['feature'].unique():
    val = test_cash.loc[i, f]
    for n, row in scorecard_cash_final[scorecard_cash_final['feature'] == f].iterrows():
      for interval in row['Value']:
        if (type(val) == str and type(interval) == list) or type(val) == float:
          if val in interval:
            score_cash[i] += row['score']

In [ ]:
for i in valid_cash.index:
  for f in scorecard_cash_final['feature'].unique():
    val = valid_cash.loc[i, f]
    for n, row in scorecard_cash_final[scorecard_cash_final['feature'] == f].iterrows():
      for interval in row['Value']:
        if (type(val) == str and type(interval) == list) or type(val) == float:
          if val in interval:
            score_cash_valid[i] += row['score']

In [ ]:
for i in test_rev.index:
  for f in scorecard_rev_final['feature'].unique():
    val = test_rev.loc[i, f]
    for n, row in scorecard_rev_final[scorecard_rev_final['feature'] == f].iterrows():
      for interval in row['Value']:
        if (type(val) == str and type(interval) == list) or type(val) == float:
          if val in interval:
            score_rev[i] += row['score']

In [ ]:
for i in valid_rev.index:
  for f in scorecard_rev_final['feature'].unique():
    val = valid_rev.loc[i, f]
    for n, row in scorecard_rev_final[scorecard_rev_final['feature'] == f].iterrows():
      for interval in row['Value']:
        if (type(val) == str and type(interval) == list) or type(val) == float:
          if val in interval:
            score_rev_valid[i] += row['score']

In [ ]:
test_cash_for_effect = pd.DataFrame(index = score_cash.index, columns=['target', 'score', 'sum'])
test_cash_for_effect['target'] = y_test_cash
test_cash_for_effect['score'] = score_cash
test_cash_for_effect['sum'] = test.loc[score_cash.index, 'AMT_CREDIT']
test_cash_for_effect.to_csv('test_cash_for_effect.csv')

valid_cash_for_effect = pd.DataFrame(index = score_cash_valid.index, columns=['target', 'score', 'sum'])
valid_cash_for_effect['target'] = y_valid_cash
valid_cash_for_effect['score'] = score_cash_valid
valid_cash_for_effect['sum'] = train.loc[score_cash_valid.index, 'AMT_CREDIT']
valid_cash_for_effect.to_csv('valid_cash_for_effect.csv')

test_rev_for_effect = pd.DataFrame(index = score_rev.index, columns=['target', 'score', 'sum'])
test_rev_for_effect['target'] = y_test_rev
test_rev_for_effect['score'] = score_rev
test_rev_for_effect['sum'] = test.loc[score_rev.index, 'AMT_CREDIT']
test_rev_for_effect.to_csv('test_rev_for_effect.csv')

valid_rev_for_effect = pd.DataFrame(index = score_rev_valid.index, columns=['target', 'score', 'sum'])
valid_rev_for_effect['target'] = y_valid_rev
valid_rev_for_effect['score'] = score_rev_valid
valid_rev_for_effect['sum'] = train.loc[score_rev_valid.index, 'AMT_CREDIT']
valid_rev_for_effect.to_csv('valid_rev_for_effect.csv')

## Экономический эффект

Считаем что кредиты берутся на 3 года под 10 процентов годовых для потребов и 20 для возобновляемых
Дифференцированный платеж (сумма долга должна уменьшаться равномерно), значит прибыль банка 0.1s+2/3 * 0.1s + 1/3 * 0.1s для потребов и 0.2s+2/3 * 0.2s + 1/3 * 0.2s для возобновляемых, причем для возоб. s=0.7*лимит (0.7 - процент утилизации)

In [ ]:
test_cash_for_effect = pd.read_csv('/content/drive/My Drive/test_cash_for_effect.csv').set_index('SK_ID_CURR')
valid_cash_for_effect = pd.read_csv('/content/drive/My Drive/valid_cash_for_effect.csv').set_index('SK_ID_CURR')
test_rev_for_effect = pd.read_csv('/content/drive/My Drive/test_rev_for_effect.csv').set_index('SK_ID_CURR')
valid_rev_for_effect = pd.read_csv('/content/drive/My Drive/valid_rev_for_effect.csv').set_index('SK_ID_CURR')

In [ ]:
valid_cash_for_effect['pred'] = 0

In [ ]:
valid_cash_for_effect

,target,score,sum,pred
SK_ID_CURR,,,,
254150,0,347,254700.0,0
161456,0,388,99504.0,0
186858,0,437,494550.0,0
149833,1,375,203760.0,0
260206,0,360,808650.0,0
...,...,...,...,...
134417,0,334,143910.0,0
438005,0,406,390447.0,0
320354,0,455,765000.0,0


In [83]:
start = valid_cash_for_effect['score'].min()
finish = np.percentile(valid_cash_for_effect['score'], 50)

In [85]:
best_profit = 0
best_treshold = start
for treshold in range(int(start), int(finish)):
  valid_cash_for_effect['pred'] = np.vectorize(lambda x: 1 if x < treshold else 0)(valid_cash_for_effect['score'])
  profit = valid_cash_for_effect[valid_cash_for_effect['target'] == 0][valid_cash_for_effect['pred'] == 0]['sum'].sum()*0.2
  profit -= valid_cash_for_effect[valid_cash_for_effect['target'] == 1][valid_cash_for_effect['pred'] == 0]['sum'].sum()
  if profit >= best_profit:
    best_profit = profit
    best_treshold = treshold
print(best_profit)
print(best_treshold)


3161448432.9000006
236


In [86]:
test_cash_for_effect['pred'] = np.vectorize(lambda x: 1 if x < best_treshold else 0)(test_cash_for_effect['score'])
profit_cash = test_cash_for_effect[test_cash_for_effect['target'] == 0][test_cash_for_effect['pred'] == 0]['sum'].sum()*0.2
profit_cash -= test_cash_for_effect[test_cash_for_effect['target'] == 1][test_cash_for_effect['pred'] == 0]['sum'].sum()
loss_cash = test_cash_for_effect[test_cash_for_effect['target'] == 0][test_cash_for_effect['pred'] == 1]['sum'].sum()*0.2
profit_cash

2816077445.1000004

In [87]:
start = valid_rev_for_effect['score'].min()
finish = np.percentile(valid_rev_for_effect['score'], 50)

In [97]:
best_profit = 0
best_treshold = start
for treshold in range(int(start), int(finish)):
  valid_rev_for_effect['pred'] = np.vectorize(lambda x: 1 if x < treshold else 0)(valid_rev_for_effect['score'])
  profit = valid_rev_for_effect[valid_rev_for_effect['target'] == 0][valid_rev_for_effect['pred'] == 0]['sum'].sum()*0.7*0.4
  profit -= valid_rev_for_effect[valid_rev_for_effect['target'] == 1][valid_rev_for_effect['pred'] == 0]['sum'].sum()*0.7
  if profit > best_profit:
    best_profit = profit
    best_treshold = treshold
print(best_profit)
print(best_treshold)


379449000.0
273


In [99]:
test_rev_for_effect['pred'] = np.vectorize(lambda x: 1 if x < best_treshold else 0)(test_rev_for_effect['score'])
profit_rev = test_rev_for_effect[test_rev_for_effect['target'] == 0][test_rev_for_effect['pred'] == 0]['sum'].sum()*0.7*0.4
profit_rev -= test_rev_for_effect[test_rev_for_effect['target'] == 1][test_rev_for_effect['pred'] == 0]['sum'].sum()*0.7
loss_rev = test_rev_for_effect[test_rev_for_effect['target'] == 0][test_rev_for_effect['pred'] == 1]['sum'].sum()*0.7*0.4
profit_rev

341406450.0

In [101]:
total_profit = profit_cash+profit_rev
total_loss = loss_cash+loss_rev
total_profit

3157483895.1000004

In [102]:
total_loss

10704541.5

## Экономический эффект для бустинга

In [108]:
boost_valid = pd.read_csv('/content/drive/My Drive/Notebooksvalid_res.csv').set_index('SK_ID_CURR').drop(columns='0')
boost_test = pd.read_csv('/content/drive/My Drive/Notebookstest_res.csv').set_index('SK_ID_CURR').drop(columns='0')

In [112]:
boost_valid.columns=['prob']
boost_test.columns=['prob']

In [110]:
train = pd.read_csv(path + '/train.csv').set_index('SK_ID_CURR')
test = pd.read_csv(path + '/test.csv').set_index('SK_ID_CURR')

In [118]:
boost_valid['target'] = train.loc[boost_valid.index, 'TARGET']
boost_test['target'] = test['TARGET']
boost_valid['sum'] = train.loc[boost_valid.index, 'AMT_CREDIT']
boost_test['sum'] = test['AMT_CREDIT']
boost_valid['type'] = train.loc[boost_valid.index, 'NAME_CONTRACT_TYPE']
boost_test['type'] = test['NAME_CONTRACT_TYPE']

In [135]:
best_profit = 0
best_treshold = 0.0
treshold = 0.0
while treshold < 0.5:
  boost_valid['pred'] = np.vectorize(lambda x: 1 if x > treshold else 0)(boost_valid['prob'])
  profit = boost_valid[boost_valid['target'] == 0][boost_valid['pred'] == 0][boost_valid['type'] == 'Revolving loans']['sum'].sum()*0.7*0.4
  profit += boost_valid[boost_valid['target'] == 0][boost_valid['pred'] == 0][boost_valid['type'] == 'Cash loans']['sum'].sum()*0.2
  profit -= boost_valid[boost_valid['target'] == 1][boost_valid['pred'] == 0][boost_valid['type'] == 'Revolving loans']['sum'].sum()*0.7
  profit -= boost_valid[boost_valid['target'] == 1][boost_valid['pred'] == 0][boost_valid['type'] == 'Cash loans']['sum'].sum()
  if profit > best_profit:
    best_profit = profit
    best_treshold = treshold
  treshold += 0.05
print(best_profit)
print(best_treshold)

3957655484.700001
0.2


In [125]:
boost_test['pred'] = np.vectorize(lambda x: 1 if x > best_treshold else 0)(boost_test['prob'])
profit_boost = boost_test[boost_test['target'] == 0][boost_test['pred'] == 0][boost_test['type'] == 'Revolving loans']['sum'].sum()*0.7*0.4
profit_boost += boost_test[boost_test['target'] == 0][boost_test['pred'] == 0][boost_test['type'] == 'Cash loans']['sum'].sum()*0.2
profit_boost -= boost_test[boost_test['target'] == 1][boost_test['pred'] == 0][boost_test['type'] == 'Revolving loans']['sum'].sum()*0.7
profit_boost -= boost_test[boost_test['target'] == 1][boost_test['pred'] == 0][boost_test['type'] == 'Cash loans']['sum'].sum()
loss_boost = boost_test[boost_test['target'] == 0][boost_test['pred'] == 1][boost_test['type'] == 'Revolving loans']['sum'].sum()*0.7*0.4
loss_boost += boost_test[boost_test['target'] == 0][boost_test['pred'] == 1][boost_test['type'] == 'Cash loans']['sum'].sum()*0.2

In [129]:
loss_boost

141154136.1

In [132]:
profit_boost

3351374100.0